In [1]:
# Imports and setup
from pydrake.all import *
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.meshcat_utils import AddMeshcatTriad
from pathlib import Path
import numpy as np
import sys

sys.path.append(str(Path.cwd()))
from throw_helpers import create_q_knots, joint_angles_to_pose

In [2]:
# Start meshcat
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
# Scene configuration
TABLE_SDF_PATH = f"{Path.cwd()}/assets/table.sdf"

# Ball position: in front of robot, on table surface
BALL_POS = [0, -0.5, 0.025]  # x=side, y=forward, z=up

scenario_yaml = f"""directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
- add_model:
    name: table
    file: file://{TABLE_SDF_PATH}
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
        translation: [0.0, 0.0, -0.05]
        rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: ball
    file: package://drake_models/manipulation_station/sphere.sdf
    default_free_body_pose:
        base_link:
            translation: {BALL_POS}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

In [4]:
# Build and visualize initial scene
meshcat.Delete()

builder = DiagramBuilder()
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant")
scene_graph = station.GetSubsystemByName("scene_graph")

# Get initial poses
temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)

X_WG_initial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("body"))
ball_body = plant.GetBodyByName("base_link", plant.GetModelInstanceByName("ball"))
X_WBall = plant.EvalBodyPoseInWorld(temp_plant_context, ball_body)

print(f"Gripper initial pose: {X_WG_initial}")
print(f"Ball pose: {X_WBall}")

# Add frame triad attached to gripper body
AddFrameTriadIllustration(
    scene_graph=scene_graph,
    body=plant.GetBodyByName("body"),
    length=0.18,
    radius=0.005,
)

# Build diagram and publish to meshcat (no simulation yet)
diagram = builder.Build()
context = diagram.CreateDefaultContext()
diagram.ForcedPublish(context)

Gripper initial pose: RigidTransform(
  R=RotationMatrix([
    [0.999999682931835, 0.00019052063137842197, -0.0007731999219133521],
    [0.0007963267107334455, -0.2392492533556365, 0.9709578572896667],
    [1.8684985011115333e-16, -0.9709581651495918, -0.23924932921398262],
  ]),
  p=[0.0003707832187589982, -0.46561680802324634, 0.6793215789060889],
)
Ball pose: RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, -0.5, 0.025],
)


In [5]:
# Pick trajectory parameters
GRIPPER_OPEN = 0.107
GRIPPER_CLOSED = 0.0
GRASP_HEIGHT = 0.12         # Height above ball center to grasp
APPROACH_HEIGHT = 0.1       # Height above grasp pose for approach

def design_grasp_pose(X_WO: RigidTransform) -> RigidTransform:
    """Grasp pose above object with gripper pointing down."""
    return X_WO @ RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), [0, 0, GRASP_HEIGHT])

def make_pick_trajectory(
    X_WG_start,
    X_WBall,
    t_approach=1.0,
    t_descend=0.5,
    t_grasp=0.5,
    t_lift=0.5,
):
    """
    Generate pick trajectory: approach -> descend -> grasp -> lift.
    Returns: (q_traj, gripper_traj, total_time, X_WG_end)
    """
    X_WG_grasp = design_grasp_pose(X_WBall)
    X_WG_approach = RigidTransform(p=[0, 0, APPROACH_HEIGHT]) @ X_WG_grasp
    
    # Keyframe times
    t0, t1, t2, t3 = 0, t_approach, t_approach + t_descend, t_approach + t_descend + t_grasp
    t4 = t3 + t_lift
    
    # Pose trajectory: start -> approach -> grasp -> hold -> lift back to approach
    pose_traj = PiecewisePose.MakeLinear(
        [t0, t1, t2, t3, t4],
        [X_WG_start, X_WG_approach, X_WG_grasp, X_WG_grasp, X_WG_approach]
    )
    
    # Gripper trajectory: open -> open -> close -> close -> close
    gripper_traj = PiecewisePolynomial.FirstOrderHold(
        [t0, t1, t2, t3, t4],
        np.array([[GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_CLOSED, GRIPPER_CLOSED]])
    )
    
    # Convert to joint trajectory via IK
    num_samples = 50
    t_samples = np.linspace(t0, t4, num_samples)
    poses = [pose_traj.GetPose(t) for t in t_samples]
    q_knots = create_q_knots(plant, poses)
    q_traj = PiecewisePolynomial.CubicShapePreserving(t_samples[:len(q_knots)], np.array(q_knots).T)
    
    return q_traj, gripper_traj, t4, X_WG_approach

In [6]:
# Generate pick trajectory
q_traj, gripper_traj, total_time, X_WG_postpick = \
    make_pick_trajectory(
        X_WG_start=X_WG_initial,
        X_WBall=X_WBall,
        t_approach=1.0,
        t_descend=0.3,
        t_grasp=0.5,
        t_lift=0.5,
    )
print(f"Pick trajectory: {total_time:.1f}s, {q_traj.get_number_of_segments()} segments")

Pick trajectory: 2.3s, 49 segments


In [7]:
# Throw parameters and joint angles
P_TARGET = np.array([0, 1.0, 0.0])  # Target position
RELEASE_FRAC = 0.5                  # Release halfway through throw
THROW_DURATION = 0.4                # Time for throw motion (s)

# Compute throw heading (direction to target)
throw_heading = np.arctan2(P_TARGET[1], P_TARGET[0])
ja1 = throw_heading - np.pi  # Joint 1 points opposite to throw direction

# Prethrow: wound up (joints 4,6 bent), Throwend: extended (joints 4,6 straight)
PRETHROW_JA = np.array([ja1, 0, 0, 1.9, 0, -1.9, np.pi])
THROWEND_JA = np.array([ja1, 0, 0, 0.4, 0, -0.4, np.pi])
release_ja = PRETHROW_JA + RELEASE_FRAC * (THROWEND_JA - PRETHROW_JA)
q_postpick = q_traj.value(q_traj.end_time()).flatten()

# Generate throw trajectory samples
def interpolate_joint_angles(ja_start, ja_end, duration, num_samples):
    t_samples = np.linspace(0, duration, num_samples, endpoint=True)
    q_samples = [ja_start + (t / duration) * (ja_end - ja_start) for t in t_samples]
    return t_samples, q_samples

t_throw_samples, q_throw_samples = interpolate_joint_angles(
    PRETHROW_JA, THROWEND_JA, THROW_DURATION, 30
)

print(f"Target: {P_TARGET}, Throw heading: {np.degrees(throw_heading):.1f}°")

Target: [0. 1. 0.], Throw heading: 90.0°


In [8]:
# Full pick + throw simulation
meshcat.Delete()
builder = DiagramBuilder()
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)
full_plant = station.GetSubsystemByName("plant")

# Add triads for key poses
AddMeshcatTriad(meshcat, "target", length=0.2, radius=0.01, X_PT=RigidTransform(p=P_TARGET))
AddMeshcatTriad(meshcat, "postpick", length=0.12, radius=0.005, X_PT=joint_angles_to_pose(full_plant, q_postpick))
AddMeshcatTriad(meshcat, "prethrow", length=0.12, radius=0.005, X_PT=joint_angles_to_pose(full_plant, PRETHROW_JA))
AddMeshcatTriad(meshcat, "release", length=0.12, radius=0.005, X_PT=joint_angles_to_pose(full_plant, release_ja))
AddMeshcatTriad(meshcat, "throwend", length=0.12, radius=0.005, X_PT=joint_angles_to_pose(full_plant, THROWEND_JA))

# Build full trajectory: pick -> prethrow -> throw
T_GO_TO_PRETHROW = 1.5
t_pick_end = q_traj.end_time()
t_full = list(np.linspace(0, t_pick_end, 50))
t_full += [t_pick_end + T_GO_TO_PRETHROW]
t_full += list(t_pick_end + T_GO_TO_PRETHROW + t_throw_samples[1:])
q_full = [q_traj.value(t).flatten() for t in np.linspace(0, t_pick_end, 50)]
q_full += [PRETHROW_JA] + q_throw_samples[1:]
q_full_traj = PiecewisePolynomial.CubicShapePreserving(t_full, np.array(q_full).T)

# Gripper: open -> close at grasp -> open at release
t_grasp = 1.3
t_release = t_pick_end + T_GO_TO_PRETHROW + RELEASE_FRAC * THROW_DURATION
g_full_traj = PiecewisePolynomial.FirstOrderHold(
    [0, t_grasp, t_grasp + 0.01, t_release, t_release + 0.01, q_full_traj.end_time()],
    np.array([[GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_CLOSED, GRIPPER_CLOSED, GRIPPER_OPEN, GRIPPER_OPEN]])
)

# Connect and run
q_source = builder.AddSystem(TrajectorySource(q_full_traj))
g_source = builder.AddSystem(TrajectorySource(g_full_traj))
builder.Connect(q_source.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(g_source.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.AdvanceTo(q_full_traj.end_time() + 2.0)
meshcat.StopRecording()
meshcat.PublishRecording()
print(f"Full trajectory: {q_full_traj.end_time():.2f}s, release at t={t_release:.2f}s")

Full trajectory: 4.20s, release at t=4.00s
